# Multi-Layer Perceptron

In [3]:
from sklearn.metrics import r2_score, max_error, mean_absolute_error, mean_absolute_percentage_error
from dataset_storing import cal_fire_temps, k_fold, fire_X, fire_centroid_x, fire_centroid_y, fire_area
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
import tensorflow as tf 
from tensorflow.keras.metrics import R2Score, MeanSquaredError, MeanAbsoluteError
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Activation
from tensorflow.keras.losses import CosineSimilarity

this is working
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'l

Predicting area

In [4]:
fire_X.head(1)

,1_MONTHS_BEFORE,2_MONTHS_BEFORE,3_MONTHS_BEFORE,4_MONTHS_BEFORE,5_MONTHS_BEFORE,6_MONTHS_BEFORE,7_MONTHS_BEFORE,8_MONTHS_BEFORE,9_MONTHS_BEFORE,10_MONTHS_BEFORE,11_MONTHS_BEFORE,12_MONTHS_BEFORE
489,"[17.88, 15.861999999999998, 15.863, 15.8619999...","[17.609, 12.479, 11.754, 12.479, 14.687, 17.60...","[14.373, 9.202, 7.236000000000001, 9.202, 10.0...","[12.751, 8.165, 5.541, 8.165, 8.391, 12.751, 8...","[13.76, 10.718, 9.472, 10.718, 10.897, 13.76, ...","[15.414, 14.002, 14.239, 14.002, 14.809, 15.41...","[14.753, 14.117, 15.045, 14.117, 14.889, 14.75...","[16.518, 16.666, 20.099, 16.666, 18.107, 16.51...","[17.834, 18.829, 23.84800000000001, 18.829, 21...","[20.307, 20.325, 26.785, 20.325, 25.265, 20.30...","[21.683000000000003, 20.601, 26.483, 20.601, 2...","[19.936, 18.781, 21.254, 18.781, 20.896, 19.93..."


In [6]:
split_indices = k_fold.split(fire_X, fire_centroid_x)
accuracy_area_dict = {
    "mean_error": [],
    "mean_sq_error": [],
    "max_error": [],
    "r2_score": []
}



for i, (train, test) in enumerate(split_indices):
    model = Sequential([ 
        # reshape 28 row * 28 column data to 28*28 rows 
        Flatten(input_shape=fire_X.shape), 
        Dense(256, activation='relu'),  
        Dense(128, activation='relu'),
        Dense(1, activation='linear'),   
    ]) 
    with CosineSimilarity() as cos_loss:
        model.compile(optimization='adam', loss=cos_loss)
    
    train_X = fire_X[train]
    train_y = fire_area.iloc[train]
    test_X = fire_X[test]
    test_y = fire_area.iloc[test]

    model.fit(train_X, train_y, epochs=100)

    

